In [1]:
import pandas as pd

In [2]:
sf = pd.read_csv('final_data2.csv')
sf.head()

,address,bathrooms,bedrooms,finishedsqft,lastsoldprice,neighborhood,totalrooms
0,Address: 1160 Mission Street #2007,2.0,2,1043,1300000,South of Market,4
1,Address: 260 King Street #475,1.0,1,903,750000,South of Market,3
2,Address: 560 Missouri Street #B,4.0,3,1425,1495000,Potrero Hill,6
3,Address: 350 Missouri Street,3.0,3,2231,2700000,Potrero Hill,10
4,Address: 3658 Folsom Street,3.0,3,1300,1530000,Bernal Heights,4


In [3]:
sf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11330 entries, 0 to 11329
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   address        11330 non-null  object 
 1   bathrooms      11330 non-null  float64
 2   bedrooms       11330 non-null  int64  
 3   finishedsqft   11330 non-null  int64  
 4   lastsoldprice  11330 non-null  int64  
 5   neighborhood   11330 non-null  object 
 6   totalrooms     11330 non-null  int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 619.7+ KB


In [4]:
sf.describe()

,bathrooms,bedrooms,finishedsqft,lastsoldprice,totalrooms
count,11330.000000,11330.000000,11330.000000,1.133000e+04,11330.000000
mean,1.980229,2.614475,1585.420918,1.263928e+06,6.111562
std,1.047358,1.299457,921.978245,1.042079e+06,12.125819
min,0.500000,0.000000,1.000000,5.350000e+02,1.000000
25%,1.000000,2.000000,1019.000000,7.292500e+05,4.000000
50%,2.000000,2.000000,1362.000000,9.900000e+05,5.000000
75%,2.000000,3.000000,1876.000000,1.450000e+06,7.000000
max,14.000000,20.000000,27275.000000,2.388900e+07,1264.000000


Because each neighborhood has the different home price, what we really want is the price per squre foot. So, we add a new variable "price_per_sqft". 

In [5]:
sf['price_per_sqft'] = sf['lastsoldprice']/sf['finishedsqft']

Also there are 71 neighborhoods in the data, we are going to group them.

In [6]:
len(sf['neighborhood'].value_counts())

71

In [7]:
freq = sf.groupby('neighborhood').count()['address']
#mean_price: average price per sqft
mean = sf.groupby('neighborhood').mean()['price_per_sqft']
cluster = pd.concat([freq, mean], axis=1)
cluster['neighborhood'] = cluster.index

In [8]:
cluster.columns = ['freq', 'price_per_sqft','neighborhood']

In [9]:
cluster.describe()

,freq,price_per_sqft
count,71.000000,71.000000
mean,159.577465,1664.908308
std,126.572696,3619.277749
min,3.000000,374.201197
25%,67.500000,613.337664
50%,123.000000,756.246284
75%,210.500000,985.156646
max,540.000000,26914.471572


Cluster the neighborhood into three groups: 1. low price, 2. high price low frequency, 3. high price high frequency

In [10]:
cluster1 = cluster[cluster.price_per_sqft < 756]
cluster1.index

Index(['Bayview', 'Central Richmond', 'Central Sunset', 'Crocker Amazon',
       'Daly City', 'Diamond Heights', 'Excelsior', 'Forest Hill',
       'Forest Hill Extension', 'Golden Gate Heights', 'Ingleside',
       'Ingleside Heights', 'Ingleside Terrace', 'Inner Parkside',
       'Inner Richmond', 'Inner Sunset', 'Lakeshore', 'Little Hollywood',
       'Merced Heights', 'Mission Terrace', 'Mount Davidson Manor',
       'Oceanview', 'Outer Mission', 'Outer Parkside', 'Outer Richmond',
       'Outer Sunset', 'Parkside', 'Portola', 'Silver Terrace', 'Sunnyside',
       'Visitacion Valley', 'West Portal', 'Western Addition',
       'Westwood Highlands', 'Westwood Park'],
      dtype='object', name='neighborhood')

In [11]:
cluster_temp = cluster[cluster.price_per_sqft >= 756]
cluster2 = cluster_temp[cluster_temp.freq <123]
cluster2.index

Index(['Buena Vista Park', 'Central Waterfront - Dogpatch', 'Corona Heights',
       'Haight-Ashbury', 'Lakeside', 'Lone Mountain', 'Midtown Terrace',
       'North Beach', 'North Waterfront', 'Parnassus - Ashbury',
       'Presidio Heights', 'Sea Cliff', 'St. Francis Wood', 'Telegraph Hill',
       'Twin Peaks'],
      dtype='object', name='neighborhood')

In [12]:
cluster3 = cluster_temp[cluster_temp.freq >=123]
cluster3.index

Index(['Bernal Heights', 'Cow Hollow', 'Downtown',
       'Eureka Valley - Dolores Heights - Castro', 'Glen Park', 'Hayes Valley',
       'Lake', 'Lower Pacific Heights', 'Marina', 'Miraloma Park', 'Mission',
       'Nob Hill', 'Noe Valley', 'North Panhandle', 'Pacific Heights',
       'Potrero Hill', 'Russian Hill', 'South Beach', 'South of Market',
       'Van Ness - Civic Center', 'Yerba Buena'],
      dtype='object', name='neighborhood')

Add group column based on the clusters

In [13]:
def get_group(x):
    if x in cluster1.index:
        return 'low_price'
    elif x in cluster2.index:
        return 'high_price_low_freq'
    else:
        return 'high_price_high_freq'
sf['group'] = sf.neighborhood.apply(get_group)

Now let's preprocess the categorical input feature, usecode

In [14]:
sf.head().T

,0,1,2,3,4
address,Address: 1160 Mission Street #2007,Address: 260 King Street #475,Address: 560 Missouri Street #B,Address: 350 Missouri Street,Address: 3658 Folsom Street
bathrooms,2.0,1.0,4.0,3.0,3.0
bedrooms,2,1,3,3,3
finishedsqft,1043,903,1425,2231,1300
lastsoldprice,1300000,750000,1495000,2700000,1530000
neighborhood,South of Market,South of Market,Potrero Hill,Potrero Hill,Bernal Heights
totalrooms,4,3,6,10,4
price_per_sqft,1246.404602,830.564784,1049.122807,1210.219632,1176.923077
group,high_price_high_freq,high_price_high_freq,high_price_high_freq,high_price_high_freq,high_price_high_freq
